In [13]:
import os
import csv
import json
import requests
import cfscrape
import traceback
from tqdm import tqdm
from time import sleep
from random import uniform
from fake_useragent import UserAgent
import numpy as np
import pandas as pd

In [5]:
# pip install cfscrape

In [6]:
# pip install requests==2.29.0

In [8]:
# pip install fake-useragent

In [ ]:
CSV_NUMBER = 'cian'  # Постфикс названия создаваемой таблицы
CSV_PATH = os.path.normpath(os.path.join(os.getcwd(), 'csv'))  # Создаём папку 'csv' для записи создаваемых таблиц
ua = UserAgent()

# Если папки нет — создаём её
if not os.path.exists(CSV_PATH):
    os.mkdir(CSV_PATH)
    print(f'Folder {CSV_PATH} has been created!')

# Словарь некоторых городов с номерами, объявления по которым можно искать на Циан
regions = {
    'msk': 1,  # Москва
}

# Названия столбцов (header) будущей таблицы,
# которые связываются с отобранными признаками в create_table()
dataset = [
    [
        'region',
        'address',
        'price',
        'total_area',
        'kitchen_area',
        'living_area',
        'rooms_count',
        'floor',
        'floors_number',
        'build_date',
        'is_complete',
        'completion_year',
        'house_material',
        'parking',
        'decoration',
        'balcony',
        'passenger_elevator',
        'cargo_elevator',
        'metro',
        'metro_distance',
        'metro_transport',
        'district',
        'is_apartments',
        'from_developer',
        'is_auction',
        'link',  # Добавляем колонку для ссылок
        'photos'  # Добавляем колонку для ссылок на фотографии
    ]
]

# Функция для обработки пропусков и булевых значений
def add_attr(attr):
    if isinstance(attr, bool):
        return int(attr)
    return attr if attr is not None else None

# Функция для создания экземпляра класса запросов
def get_session():
    headers = {
        'authority': 'www.cian.ru',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        # 'cookie': '_CIAN_GK=ba9cbcd7-318e-466f-acc9-76edbde6f3e8; _gcl_au=1.1.547104461.1710680659; tmr_lvid=57770ba711bf55b7a32e568f304f9f9f; tmr_lvidTS=1710680659331; login_mro_popup=1; sopr_utm=%7B%22utm_source%22%3A+%22direct%22%2C+%22utm_medium%22%3A+%22None%22%7D; uxfb_usertype=searcher; sopr_session=4690d323f46b4532; _gid=GA1.2.1804219744.1710680661; _ym_uid=1710680662970774452; _ym_d=1710680662; uxs_uid=df351200-e45e-11ee-85dd-e90cfbd1a20d; afUserId=f76d2a6f-9a78-48ed-8085-df3d3715ca3d-p; AF_SYNC=1710680661868; _ym_isad=2; _ym_visorc=b; adrdel=1; adrcid=AT0UT0wEPNDp1rk7PPeLAzg; session_region_id=1; session_main_town_region_id=1; __cf_bm=BbPuZP1Voe0Xuq68MmgPcpmXtot4vjcdiWJ1urjxOfw-1710681351-1.0.1.1-iLJsaBY6woS.xbe4OQIJPoqgWhSmKSUl1JipA4iphNKsAWiXU.WTC0SqaeNxX_1Lxz9uQGoa8WKXCzkHS53_2Q; _ga_3369S417EL=GS1.1.1710680661.1.1.1710681355.60.0.0; _ga=GA1.2.1802668360.1710680661; _dc_gtm_UA-30374201-1=1; tmr_detect=0%7C1710681357609',
        'referer': 'https://www.cian.ru/',
        'sec-ch-ua': '"Not(A:Brand";v="24", "Chromium";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
    }

    session = requests.Session()
    session.headers = headers
    return cfscrape.create_scraper(sess=session)  # cfscrape — обход защиты от ботов Cloudflare

# Записываем всё в файл формата .csv
def recording_table():
    try:
        with open(os.path.join(CSV_PATH, f'data_{CSV_NUMBER}.csv'), mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            for row in dataset:
                writer.writerow(row)
        print(f'The dataset is written in file "data_{CSV_NUMBER}.csv"')
    except Exception as error:
        print('Recording error!\n', traceback.format_exc())

# Получаем формат json (питоновский dict) из нашего запроса Response
def get_json(session, region_name, cur_page):
    json_data = {
        'jsonQuery': {
            '_type': 'flatsale',
            'engine_version': {'type': 'term', 'value': 2},
            'region': {'type': 'terms', 'value': [regions[region_name]]},
            'room': {'type': 'terms', 'value': [1, 2, 3, 4, 5, 6]},
            'page': {'type': 'term', 'value': cur_page},
        }
    }

    try:
        response = session.post('https://api.cian.ru/search-offers/v2/search-offers-desktop/', json=json_data)
    except:
        return f'oops! Error {response.status_code}'

    if response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json"):
        try:
            return response.json()
        except ValueError:
            return f'oops! ValueError!'

# Создание таблицы и добавление ссылок
def create_table(region_name='msk', start_page=1, end_page=55, number_of_samples=2000):
    if start_page < 1:
        start_page = 1
    if end_page > 55:
        end_page = 55

    session = get_session()

    cnt_samples = 0
    for cur_page in tqdm(range(start_page, end_page)):
        if cnt_samples >= number_of_samples:
            break


      # Случайное время паузы перед каждой отправкой запроса
        sleep_time = uniform(25, 35)  # Случайное время паузы между 25 и 35 секундами
        sleep(sleep_time)

        data = get_json(session, region_name, cur_page)
        if data is None:
            print('oops! Captcha!')
            return
        if isinstance(data, str):
            continue

        for item in data['data']['offersSerialized']:
            # Получаем ссылку на объявление
            link = item.get('fullUrl', None)
            if link:
                    link = f"{link}"

            # Получаем ссылки на фотографии
            photos = item.get('photos', [])
            photo_links = [photo['fullUrl'] for photo in photos if 'fullUrl' in photo]

            # Преобразуем список ссылок на фото в строку
            photo_links_str = ', '.join(photo_links)

            cur_item = [
                region_name,
                add_attr(item["geo"]["userInput"]),
                add_attr(item['bargainTerms']['priceRur']),
                add_attr(item.get('totalArea')),
                add_attr(item.get('kitchenArea')),
                add_attr(item.get('livingArea')),
                add_attr(item.get('roomsCount')),
                add_attr(item.get('floorNumber')),
                add_attr(item['building'].get('floorsCount')),
                add_attr(item['building'].get('buildYear')),
                add_attr(item['building']['deadline']['isComplete'] if item['building'].get('deadline') is not None else None),
                add_attr(item['building']['deadline']['year'] if item['building'].get('deadline') is not None else None),
                add_attr(item['building'].get('materialType')),
                add_attr(item['building']['parking']['type'] if item['building'].get('parking') is not None else None),
                add_attr(item.get('decoration')),
                add_attr(item.get('balconiesCount')),
                add_attr(item['building'].get('passengerLiftsCount')),
                add_attr(item['building'].get('cargoLiftsCount')),
                add_attr(','.join([str(x['name']) for x in item['geo']['undergrounds'] if x is not None])),
                add_attr(','.join([str(x['time']) for x in item['geo']['undergrounds'] if x is not None])),
                add_attr(','.join([str(x['transportType']) for x in item['geo']['undergrounds'] if x is not None])),
                add_attr(','.join([str(x['name']) for x in item['geo']['districts'] if x is not None])),
                add_attr(item.get('isApartments')),
                add_attr(item.get('fromDeveloper')),
                add_attr(item.get('isAuction')),
                add_attr(link),  # Ссылка на объявление
                add_attr(photo_links_str)  # Добавляем ссылки на фотографии
            ]

            if cur_item not in dataset:
                dataset.append(cur_item)
                cnt_samples += 1
            else:
                continue

            if cnt_samples >= number_of_samples:
                break

        print(f'{cnt_samples} / {number_of_samples} | page: {cur_page}')

    recording_table()
    return

# Запуск функции
create_table()


  2%|▏         | 1/54 [00:30<27:02, 30.62s/it]

28 / 2000 | page: 1


  4%|▎         | 2/54 [00:56<23:54, 27.59s/it]

56 / 2000 | page: 2


  6%|▌         | 3/54 [01:24<23:48, 28.02s/it]

...

1005 / 2000 | page: 36


 69%|██████▊   | 37/54 [16:31<07:11, 25.37s/it]

1033 / 2000 | page: 37


 70%|███████   | 38/54 [16:59<06:56, 26.02s/it]

1061 / 2000 | page: 38
